### Ivadomed Tutorial 3: **Uncertainty Estimation**

This tutorial shows how to estimate uncertainty measures (aleatoric and epistemic) on the model's predictions. These uncertainty measures are already implemented in `ivadomed` and are detailed in [Technical features](https://ivadomed.org/technical_features.html#uncertainty-measures).

⚠️ Before getting started, please ensure that you: 

1.   Are connected to the GPU. You can do this by doing the following from the task bar on the top: `Runtime` $\to$ `Change Runtime type` $\to$ `Hardware accelerator: GPU`
2.   **Are running this tutorial from _your_ Google Drive. You can do this by going to: `File` $\to$ `Save a Copy in Drive`.**

In [ ]:
# @title Fetch configuration file
# fetch the configuration (config) file to be used for this tutorial
!wget https://raw.githubusercontent.com/ivadomed/ivadomed/master/ivadomed/config/config.json ./content

In [ ]:
# @title Install Dependencies
!pip install imgaug==0.2.5 --quiet 
!pip install ivadomed --quiet 

%load_ext tensorboard

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.6 MB/s eta 0:12:08tcmalloc: large alloc 1147494400 bytes == 0x5624825fa000 @  0x7fc3b5af8615 0x56244888e4cc 0x56244896e47a 0x5624488912ed 0x562448982e1d 0x562448904e99 0x5624488ff9ee 0x562448892bda 0x562448904d00 0x5624488ff9ee 0x562448892bda 0x562448901737 0x562448983c66 0x562448900daf 0x562448983c66 0x562448900daf 0x562448983c66 0x562448900daf 0x562448893039 0x5624488d6409 0x562448891c52 0x562448904c25 0x5624488ff9ee 0x562448892bda 0x562448901737 0x5624488ff9ee 0x562448892bda 0x562448900915 0x562448892afa 0x562448900c0d 0x5624488ff9ee
     |█████████████████               | 1055.7 MB 1.7 MB/s eta 0:09:00tcmalloc: large alloc 1434370048 bytes == 0x5624c6c50000 @  0x7fc3b5af8615 0x56244888e4cc 0x56244896e47a 0x5624488912ed 0x562448982e1d 0x562448904e99 0x5624488ff9ee 0x562448892bda 0x562448904d00 0x5624488ff9ee 0x562448892bda 0x562448901737 0x562448983c66 0x562448900d

In [ ]:
# @title Run Me for Downloading the Dataset!

# @markdown We will be using a publicly-available dataset consisting of the MRI data of the spinal cord. 
# @markdown More details on this dataset can be found in 
# @markdown Tutorial 1: [One-class segmentation with 2D U-Net](https://ivadomed.org/tutorials/one_class_segmentation_2d_unet.html).

# download the dataset
!ivadomed_download_data -d data_example_spinegeneric

2021-11-18 17:08:58.880 | INFO     | ivadomed.utils:init_ivadomed:408 - 
ivadomed (2.9.0)

Trying URL: https://github.com/ivadomed/data_example_spinegeneric/archive/r20200825.zip
Downloading: data_example_spinegeneric-r20200825.zip
Unzip data to: /tmp/tmpl29u9rom
Removing temporary folders...
Folder Created: /content/data_example_spinegeneric
--2021-11-18 17:09:41--  https://raw.githubusercontent.com/ivadomed/ivadomed/master/ivadomed/config/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3699 (3.6K) [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]   3.61K  --.-KB/s    in 0s      

2021-11-18 17:09:41 (43.6 MB/s) - ‘config.json’ saved [3699/3699]

--2021-11-18 17:09:41--  http://./content
Resolving . (.)... fai

#### Configuration File

In this tutorial, we will be using the configuration file: `ivadomed/config/config.json`. This is already downloaded for you and can be seen under Colab's `Files` (📁) tab on the left.

Open this file; this is the same configuration file used in the [first tutorial](https://ivadomed.org/tutorials/one_class_segmentation_2d_unet.html) and will be modified as mentioned in the [Technical features](https://ivadomed.org/technical_features.html#uncertainty-measures). Please ensure that the `path_data` key points to the correct location of the dataset. The parameters that are of interest for this tutorial are as follows: 

1. `path_data` -  Location of the directory containing the dataset. 
```json
    "path_data": "data_example_spinegeneric"
```

2. `path_output` -  Location of the directory containing the trained model. To avoid having to train a model from scratch, there is a pre-trained model for spinal cord segmentation in the folder named trained_model, in the downloaded dataset. Modify the path so it points to the location of the trained model.
```json
    "path_output": "<PATH_TO_DATASET>/data_example_spinegeneric/trained_model"
```

3. `command` - The task to perform. Since we are interested in inference on a trained model, we set the command to "test" as shown below. 
```json
    "command": "test"
```

4. `uncertainty` - The type of uncertainty to estimate. Available choices are "epistemic" and "aleatoric". Note that both can be true. More details on the implementation can be found in [Technical features](https://ivadomed.org/technical_features.html#uncertainty-measures). `"n_it"` controls the number of Monte Carlo iterations that are performed to estimate the uncertainty. It is set to a positive integer for this tutorial (e.g. `15`).
```json
    "uncertainty": {
        "epistemic": true,
        "aleatoric": true,
        "n_it": 15
    }
```

5. `transformation` - The transformations performed as a part of data augmentation. If aleatoric uncertainty is enabled, the data augmentation that will be performed is the same as the one performed for the training. Note that only transformations for which an `undo_transform` (i.e. inverse transformation) is available will be performed since these inverse transformations are required to reconstruct the predicted volume.



### Modify the Config File

Open the `config.json` file under the "Files" tab on the left. This should let you edit the contents of the json file as mentioned above. Change the following parameters:

1. 
```json
    "command": "test"
```
2. 
```json
    "path_output": "data_example_spinegeneric/trained_model"
```
3. 
```json
    "debugging": true
```
4. 
```json
    "uncertainty": {
        "epistemic": true,
        "aleatoric": true,
        "n_it": 15
    }
```

In [ ]:
# @title Run Uncertainty Estimation

# @markdown Once the configuration file has been modified, run the inference with the following command:
# @markdown ```shell
# @markdown   ivadomed --test -c config.json
# @markdown ```

# run uncertainty estimation
!ivadomed --test -c config.json

2021-11-18 17:11:09.612 | INFO     | ivadomed.utils:init_ivadomed:408 - 
ivadomed (2.9.0)

2021-11-18 17:11:09.615 | INFO     | ivadomed.utils:get_path_output:371 - CLI flag --path-output not used to specify output directory. Will check config file for directory...
2021-11-18 17:11:09.615 | INFO     | ivadomed.utils:get_path_data:383 - CLI flag --path-data not used to specify BIDS data directory. Will check config file for directory...
2021-11-18 17:11:09.615 | INFO     | ivadomed.main:set_output_path:198 - Output path already exists: data_example_spinegeneric/trained_model
2021-11-18 17:11:09.720 | INFO     | ivadomed.utils:define_device:135 - Using GPU ID 0
2021-11-18 17:11:09.721 | INFO     | ivadomed.utils:display_selected_model_spec:145 - Selected architecture: Unet, with the following parameters:
2021-11-18 17:11:09.721 | INFO     | ivadomed.utils:display_selected_model_spec:148 - 	dropout_rate: 0.3
2021-11-18 17:11:09.722 | INFO     | ivadomed.utils:display_selected_model_spec:1

If aleatoric uncertainty was enabled, then data augmentation operations will be performed at the test time, as indicated in the terminal output (see below). Note that `ElasticTransform` has been deactivated because `undo_transform` function is not available for it.

```
  Selected transformations for the ['testing'] dataset:
   Resample: {'hspace': 0.75, 'wspace': 0.75, 'dspace': 1}
   CenterCrop: {'size': [128, 128]}
   RandomAffine: {'degrees': 5, 'scale': [0.1, 0.1], 'translate': [0.03, 0.03], 'applied_to': ['im', 'gt']}
   ElasticTransform: {'alpha_range': [28.0, 30.0], 'sigma_range': [3.5, 4.5], 'p': 0.1, 'applied_to': ['im', 'gt']}
   NumpyToTensor: {}
   NormalizeInstance: {'applied_to': ['im']}
  ElasticTransform transform not included since no undo_transform available for it.
```

.... otherwise, only preprocessing and data normalization are performed, see below:

```
  Selected transformations for the ['testing'] dataset:
   Resample: {'hspace': 0.75, 'wspace': 0.75, 'dspace': 1}
   CenterCrop: {'size': [128, 128]}
   NumpyToTensor: {}
   NormalizeInstance: {'applied_to': ['im']}
```

For each testing image, `"n_it"` Monte Carlo samples for that image are segmented using the trained model and saved under `pred_masks`, with the iteration number as suffix (e.g. `sub-001_pred_00.nii.gz … sub-001_pred_19.nii.gz`).

```
  Computing model uncertainty over 20 iterations.
    Inference - Iteration 0: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.27s/it]     
    Inference - Iteration 1: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.81s/it]
    Inference - Iteration 2: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.96s/it]
    Inference - Iteration 3: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.66s/it]
    Inference - Iteration 4: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.69s/it]
    Inference - Iteration 5: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.92s/it]
    Inference - Iteration 6: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.74s/it]
    Inference - Iteration 7: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.74s/it]
    Inference - Iteration 8: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.83s/it]
    Inference - Iteration 9: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.59s/it]
    Inference - Iteration 10: 100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.85s/it]
    Inference - Iteration 11: 100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.85s/it]
    Inference - Iteration 12: 100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.92s/it]
    Inference - Iteration 13: 100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.83s/it]
    Inference - Iteration 14: 100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.84s/it]
```

The Monte Carlo samples are then used to compute uncertainty measures for each image. The results are saved under `pred_masks`.
```
  Uncertainty Computation: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [01:31<00:00, 18.28s/it]
```

In [ ]:
# @title Save and Download the results!
# @markdown Now that we have the uncertainty estimates, we would like to download
# @markdown the results locally for further anaylsis. For that, we first
# @markdown create a `.zip`file of the results folder and then download
# @markdown the zipped file manually. 

# first, zip the results folder
!zip -r --quiet spineGeneric_unc.zip ./data_example_spinegeneric/trained_model/
print("Zip file created!")

Zip file created!


Note that six files are generated during this process for each testing image:

* `*_soft.nii.gz`: Soft segmentation (i.e. values between 0 and 1) which is generated by averaging the Monte Carlo samples.

* `*_pred.nii.gz`: Binary segmentation obtained by thresholding `*_soft.nii.gz` with `1 / (Number of Monte Carlo iterations)` i.e. `1/n_it`.

* `*_unc-vox.nii.gz`: Voxel-wise measure of uncertainty derived from the entropy of the Monte Carlo samples. The higher a given voxel value is, the more uncertain is the prediction for this voxel.

* `*_unc-avgUnc.nii.gz`: Structure-wise measure of uncertainty derived from the mean value of `*_unc-vox.nii.gz` within a given connected object (e.g. a lesion, grey matter).

* `*_unc-cv.nii.gz`: Structure-wise measure of uncertainty derived from the coefficient of variation of the volume of a given connected object across the Monte Carlo samples. The higher the value for a given voxel, the more uncertain is the prediction for this voxel.

* `*_unc-iou.nii.gz`: Structure-wise measure of uncertainty derived from the Intersection-over-Union (IoU) of the predictions of a given connected object across the Monte Carlo samples. The lower the value for a given voxel, the more uncertain is the prediction for this voxel.

These files can further be used for post-processing to refine the segmentation. For example, the voxels depicted in pink under the "Uncertainty" panel are more uncertain than the ones in blue: therefore, we can further refine the model's prediction by removing the voxels with low uncertainty (in blue) **AND** low prediction values (in dark red under the "Model Prediction" panel) from the foreground class.  

<img src="https://raw.githubusercontent.com/ivadomed/doc-figures/main/tutorials/uncertainty/uncertainty_tutorial.png">


And that concludes the tutorial on how to use the in-built uncertainty estimation measures in `ivadomed` for spinal cord segmentation.